## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/MV/present"

In [4]:
waves = np.load(path + "/present_MV_waves_10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(1080, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.027625, 0.047125, 0.03375 , 0.041125]])

In [8]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [9]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [10]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [11]:
type(zcrs[0])

numpy.ndarray

In [12]:
np.save(path + "/present_MV_zcrs_array", zcrs, allow_pickle= True)

In [13]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [14]:
zcrs[7]

array([2.7625e-02, 4.7125e-02, 3.3750e-02, 4.1125e-02, 1.8060e+03])

In [15]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [16]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [17]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.025125,0.021500,0.030125,0.037375,1230.0
1,0.018750,0.045750,0.022125,0.042125,1410.0
2,0.020000,0.037125,0.031375,0.030375,1269.0
3,0.016000,0.037125,0.035000,0.032750,1404.0
4,0.021625,0.047750,0.024000,0.032875,1432.0


In [18]:
df.to_csv(path + '/present_MV_zcrs.csv')

## with Özgün Ozan Nacitarhan code

In [19]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [20]:
zcrs_özgün = zcr(waves)

In [21]:
zcrs_özgün.shape

(1080, 1)

In [22]:
zcrs_özgün[0]

array([0.0307375])

In [23]:
np.save(path + "/present_MV_zcrs_özgün", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import librosa.display

In [26]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [27]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [28]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [31]:
type(chro[0])

numpy.ndarray

In [32]:
chro.shape

(1080, 12)

In [33]:
np.save(path + "/present_MV_chromogram_array", chro, allow_pickle= True)

In [34]:
print(chro.shape)

(1080, 12)


In [35]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [36]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [37]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.810446,0.725619,0.853265,0.759749,0.702425,0.68703,0.561408,0.668863,0.736252,0.648418,0.732366,0.805493
1,0.738709,0.66695,0.640127,0.678613,0.631114,0.550491,0.62001,0.696551,0.636922,0.503559,0.450244,0.689209
2,0.593778,0.59127,0.515877,0.447003,0.398011,0.426973,0.640992,0.674925,0.683793,0.621001,0.683612,0.644247
3,0.659662,0.480534,0.405657,0.313199,0.254617,0.511502,0.540043,0.593592,0.746334,0.834766,0.705766,0.781013
4,0.721779,0.665788,0.742019,0.611769,0.472817,0.37392,0.511763,0.694966,0.71466,0.605556,0.725241,0.764058
...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.732905,0.613481,0.561687,0.5397,0.497762,0.529659,0.572538,0.513017,0.503343,0.546078,0.699644,0.813053
1076,0.66974,0.493809,0.499756,0.606515,0.628215,0.611097,0.68244,0.643369,0.618319,0.621708,0.58232,0.678999
1077,0.696522,0.5674,0.546663,0.496339,0.487712,0.446459,0.6237,0.67567,0.676504,0.653285,0.68114,0.78204
1078,0.679279,0.729566,0.756466,0.80642,0.722449,0.622377,0.524794,0.51073,0.62642,0.633286,0.602839,0.659564


In [38]:
ch_df.to_csv(path + "/present_MV_chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [39]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [40]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [41]:
tempogram(waves[0]).shape

(192, 79)

In [42]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [43]:
tempogram(waves[0])[1]

array([0.76697168, 0.76780304, 0.76866028, 0.76954996, 0.77047845,
       0.77145178, 0.7724756 , 0.7735551 , 0.77469497, 0.7758993 ,
       0.7771716 , 0.77851472, 0.77993086, 0.78142151, 0.78298755,
       0.78462915, 0.78634585, 0.78813662, 0.78999981, 0.79193327,
       0.79393437, 0.79600007, 0.79812695, 0.80031128, 0.80254909,
       0.8048362 , 0.8071683 , 0.809541  , 0.81194986, 0.81439044,
       0.81685835, 0.81934929, 0.82185905, 0.82438357, 0.82691894,
       0.82946146, 0.83200757, 0.83455396, 0.83709749, 0.83963525,
       0.84216454, 0.84468288, 0.84718799, 0.84967779, 0.8521504 ,
       0.85460414, 0.85703749, 0.85944913, 0.86183789, 0.86420274,
       0.86654283, 0.86885741, 0.87114588, 0.87340774, 0.8756426 ,
       0.87785017, 0.88003027, 0.88218276, 0.88430761, 0.88640485,
       0.88847455, 0.89051687, 0.89253199, 0.89452014, 0.8964816 ,
       0.89841666, 0.90032567, 0.90220897, 0.90406695, 0.9059    ,
       0.90770854, 0.90949299, 0.91125378, 0.91299137, 0.91470

In [44]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  8.40228357e-01,  6.08630920e-01,  5.12123698e-01,
        5.48737967e-01,  5.70647436e-01,  5.66876969e-01,  5.55998081e-01,
        5.51088565e-01,  5.55084199e-01,  5.62282720e-01,  5.49057937e-01,
        5.36770563e-01,  5.32411104e-01,  5.20952294e-01,  4.79496649e-01,
        4.45297454e-01,  4.44086376e-01,  4.47973800e-01,  4.27533437e-01,
        4.01738878e-01,  4.32863124e-01,  4.85593345e-01,  4.76399075e-01,
        4.04773923e-01,  3.49259322e-01,  3.21703390e-01,  2.96517761e-01,
        2.90563659e-01,  3.10238118e-01,  3.18295728e-01,  3.18706134e-01,
        3.31155100e-01,  3.41437779e-01,  3.18526873e-01,  2.82496840e-01,
        2.60821357e-01,  2.61799373e-01,  2.93630652e-01,  3.47790582e-01,
        3.39123733e-01,  2.72804139e-01,  2.18389561e-01,  2.37099292e-01,
        2.69134998e-01,  2.79493610e-01,  2.70815914e-01,  2.67345481e-01,
        2.62436054e-01,  2.43679666e-01,  2.30044855e-01,  2.38680324e-01,
        2.48176450e-01,  

# tempogram(waves[0])[0]

In [45]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [46]:
temp = temp_values(waves) #çok uzun sürüyor 


In [47]:
type(temp)

numpy.ndarray

In [48]:
np.save(path + "/present_MV_tempgram_array", temp, allow_pickle= True)

In [49]:
temp.shape

(1080, 192)

In [50]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [51]:
len(columns_t)

192

In [52]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.840228,0.608631,0.512124,0.548738,0.570647,0.566877,0.555998,0.551089,0.555084,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.804058,0.510143,0.400774,0.405276,0.410899,0.379941,0.362436,0.385077,0.394141,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,1.0,0.79081,0.484828,0.338964,0.352767,0.443668,0.441545,0.380033,0.365459,0.39275,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.730569,0.33581,0.110327,0.141231,0.182817,0.191808,0.123467,0.108056,0.110708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.844242,0.605675,0.416162,0.309638,0.255187,0.249018,0.253186,0.285578,0.384112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1.0,0.784627,0.398328,0.168004,0.142585,0.211737,0.297695,0.377897,0.415552,0.369398,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1076,1.0,0.832464,0.558903,0.381477,0.33686,0.34052,0.343291,0.340368,0.313565,0.275574,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1077,1.0,0.835764,0.546349,0.3319,0.277453,0.319119,0.363158,0.379023,0.370616,0.35433,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1078,1.0,0.801043,0.435805,0.2035,0.166323,0.243787,0.315055,0.303972,0.236859,0.174684,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [53]:
df_t['tempogram_188'][10]

1.3255613978690745e-17

In [54]:
df_t.to_csv(path + '/present_MV_tempogram.csv')

In [55]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [56]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.025125,0.021500,0.030125,0.037375,1230.0,0.810446,0.725619,0.853265,0.759749,0.702425,0.68703,0.561408,0.668863,0.736252,0.648418,0.732366,0.805493
1,0.018750,0.045750,0.022125,0.042125,1410.0,0.738709,0.66695,0.640127,0.678613,0.631114,0.550491,0.62001,0.696551,0.636922,0.503559,0.450244,0.689209
2,0.020000,0.037125,0.031375,0.030375,1269.0,0.593778,0.59127,0.515877,0.447003,0.398011,0.426973,0.640992,0.674925,0.683793,0.621001,0.683612,0.644247
3,0.016000,0.037125,0.035000,0.032750,1404.0,0.659662,0.480534,0.405657,0.313199,0.254617,0.511502,0.540043,0.593592,0.746334,0.834766,0.705766,0.781013
4,0.021625,0.047750,0.024000,0.032875,1432.0,0.721779,0.665788,0.742019,0.611769,0.472817,0.37392,0.511763,0.694966,0.71466,0.605556,0.725241,0.764058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.007625,0.022750,0.014500,0.015625,692.0,0.732905,0.613481,0.561687,0.5397,0.497762,0.529659,0.572538,0.513017,0.503343,0.546078,0.699644,0.813053
1076,0.012375,0.020500,0.016500,0.014500,693.0,0.66974,0.493809,0.499756,0.606515,0.628215,0.611097,0.68244,0.643369,0.618319,0.621708,0.58232,0.678999
1077,0.012125,0.017500,0.012750,0.017500,690.0,0.696522,0.5674,0.546663,0.496339,0.487712,0.446459,0.6237,0.67567,0.676504,0.653285,0.68114,0.78204
1078,0.010500,0.017250,0.017250,0.016375,686.0,0.679279,0.729566,0.756466,0.80642,0.722449,0.622377,0.524794,0.51073,0.62642,0.633286,0.602839,0.659564


In [57]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.025125,0.021500,0.030125,0.037375,1230.0,0.810446,0.725619,0.853265,0.759749,0.702425,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.018750,0.045750,0.022125,0.042125,1410.0,0.738709,0.66695,0.640127,0.678613,0.631114,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.020000,0.037125,0.031375,0.030375,1269.0,0.593778,0.59127,0.515877,0.447003,0.398011,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.016000,0.037125,0.035000,0.032750,1404.0,0.659662,0.480534,0.405657,0.313199,0.254617,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.021625,0.047750,0.024000,0.032875,1432.0,0.721779,0.665788,0.742019,0.611769,0.472817,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.007625,0.022750,0.014500,0.015625,692.0,0.732905,0.613481,0.561687,0.5397,0.497762,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1076,0.012375,0.020500,0.016500,0.014500,693.0,0.66974,0.493809,0.499756,0.606515,0.628215,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1077,0.012125,0.017500,0.012750,0.017500,690.0,0.696522,0.5674,0.546663,0.496339,0.487712,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1078,0.010500,0.017250,0.017250,0.016375,686.0,0.679279,0.729566,0.756466,0.80642,0.722449,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [58]:
final_df.to_csv(path + '/present_MV_final_sp.csv')